In [10]:
import random, csv
import numpy as np
import cPickle
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk
import networkx as nx

### Reading test and train data, and node info

In [11]:
#nltk.download('punkt') # for tokenization
#nltk.download('stopwords')
stpwds = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()
stemmer_new = nltk.stem.SnowballStemmer("english")
with open("testing_set.txt", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)

testing_set = [element[0].split(" ") for element in testing_set]

In [12]:
with open("training_set.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

training_set = [element[0].split(" ") for element in training_set]

In [13]:
with open("node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)

IDs = [element[0] for element in node_info]

In [14]:
file=open("features_train_unscaled.csv",'r')
lines=file.readlines()
feat = []
counter = 0
collab = {}
cite = {}
from collections import defaultdict
for line in lines[1:]:
    line=line.strip().split(",")  
    feat.append(line)
    source = line[0]
    target = line[1]
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    #source_title = source_info[4].lower().split(".")
    authors_A = source_info[3].split(',')
    authors_B = target_info[3].split(',')
    
    # citations
    for auth_a in authors_A:
        a = auth_a.strip()
        for auth_b in authors_B:
            b = auth_b.strip()
            if a == '' or b == '':
                continue
            if a not in cite:
                cite[a] = defaultdict(int)
            cite[a][b] += 1
    
    # collaborations
    for auth_1 in authors_A:
        a = auth_1.strip()
        for auth_2 in authors_A:
            b = auth_2.strip()
            if a == '' or b == '':
                continue
            if a not in collab:
                collab[a] = defaultdict(int)
            collab[a][b] += 1
            
    for auth_1 in authors_B:
        a = auth_1.strip()
        for auth_2 in authors_B:
            b = auth_2.strip()
            if a == '' or b == '':
                continue
            if a not in collab:
                collab[a] = defaultdict(int)
            collab[a][b] += 1
    
    counter += 1
    if counter % 20000 == True:
        print counter, "training examples processsed"

1 training examples processsed
20001 training examples processsed
40001 training examples processsed
60001 training examples processsed
80001 training examples processsed
100001 training examples processsed
120001 training examples processsed
140001 training examples processsed
160001 training examples processsed
180001 training examples processsed
200001 training examples processsed
220001 training examples processsed
240001 training examples processsed
260001 training examples processsed
280001 training examples processsed
300001 training examples processsed
320001 training examples processsed
340001 training examples processsed
360001 training examples processsed
380001 training examples processsed
400001 training examples processsed
420001 training examples processsed
440001 training examples processsed
460001 training examples processsed
480001 training examples processsed
500001 training examples processsed
520001 training examples processsed
540001 training examples processsed
5

In [17]:
# TO WRITE DICTIONARIES TO A FILE

#import json
#with open('cite.txt', 'w+') as f:
#    json.dump(cite, f)
#import json
#with open('collab.txt', 'w+') as f:
#    json.dump(collab, f)

In [3]:
# TO LOAD DICTIONARIES FROM FILE

#with open('collab.txt') as f:
#    collab = json.load(f)
#with open('cite.txt') as f:
#    cite = json.load(f)

NameError: name 'json' is not defined

### Creating graph (nodes and edges from training data)

In [19]:
edges = [(element[0],element[1]) for element in training_set if element[2]=="1"]
nodes = [element[0] for element in training_set] + [element[1] for element in training_set]

# this is the actual directed graph
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

# undirected version of the graph (used for jaccard similarity)
G_undirected = nx.Graph()
G_undirected.add_nodes_from(nodes)
G_undirected.add_edges_from(edges)

### Creating features for the entire data to store in csv file

In [13]:
### creating features and store in a csv file

max_collab_A_B = [] # max collaboration between authors of A and B
mean_collab_A_B = []

max_cite_A_B = [] # max citation by authors of A to B
mean_cite_A_B = []

#avg_cite_A_1 = [] # average citations authors of source have received
avg_cite_A_2 = [] # average citations authors of source have made

#avg_cite_B_1 = [] # average citations authors of target have received
avg_cite_B_2 = [] # average citations authors of target have made

indegree = []
counter = 0
for i in xrange(len(training_set)):
    source = training_set[i][0]
    target = training_set[i][1]
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    authors_A = source_info[3].split(',')
    authors_B = target_info[3].split(',')
    
    max_collab = []
    max_cite = []
    for auth_a in authors_A:
        a = auth_a.strip()
        for auth_b in authors_B:
            b = auth_b.strip()
            if a == '' or b == '':
                continue
            max_collab.append(collab[a][b])
            max_cite.append(cite[a][b])
            
    max_collab_A_B.append(0 if not len(max_collab) else max(max_collab))
    mean_collab_A_B.append(0 if not len(max_collab) else sum(max_collab)/len(max_collab))
    
    max_cite_A_B.append(0 if not len(max_cite) else max(max_cite) )
    mean_cite_A_B.append(0 if not len(max_cite) else sum(max_cite)/len(max_cite))
    
    avg_cite = []
    for auth_a in authors_A:
        a = auth_a.strip()
        if a == '' or not a in cite:
            continue
        avg_cite.append(sum(cite[a].values()))
    avg_cite_A_2.append(0 if not len(avg_cite) else sum(avg_cite)/len(avg_cite))
    
    avg_cite = []
    for auth_a in authors_B:
        a = auth_a.strip()
        if a == '' or not a in cite:
            continue
        avg_cite.append(sum(cite[a].values()))
    avg_cite_B_2.append(0 if not len(avg_cite) else sum(avg_cite)/len(avg_cite))
    
    indegree.append(G.in_degree(target)/len(authors_B))
    counter += 1
    if counter % 20000 == True:
        print counter, "training examples processsed"

1 training examples processsed
20001 training examples processsed
40001 training examples processsed
60001 training examples processsed
80001 training examples processsed
100001 training examples processsed
120001 training examples processsed
140001 training examples processsed
160001 training examples processsed
180001 training examples processsed
200001 training examples processsed
220001 training examples processsed
240001 training examples processsed
260001 training examples processsed
280001 training examples processsed
300001 training examples processsed
320001 training examples processsed
340001 training examples processsed
360001 training examples processsed
380001 training examples processsed
400001 training examples processsed
420001 training examples processsed
440001 training examples processsed
460001 training examples processsed
480001 training examples processsed
500001 training examples processsed
520001 training examples processsed
540001 training examples processsed
5

In [2]:
train_features = np.genfromtxt('features_train_unscaled.csv', delimiter=',')[1:]

NameError: name 'np' is not defined

In [14]:
X = np.array([max_collab_A_B, mean_collab_A_B, max_cite_A_B, mean_cite_A_B, avg_cite_A_2, avg_cite_B_2, indegree]).T
print X.shape, X[0].shape, X[0]

(615512L, 7L) (7L,) [  0   0   0   0   0 244   2]


In [19]:
new_features = np.array([max_collab_A_B, mean_collab_A_B, max_cite_A_B, mean_cite_A_B, avg_cite_A_2, avg_cite_B_2, indegree]).T
train_features = np.genfromtxt('features_train_unscaled3.csv', delimiter=',')

In [51]:
# # convert list of lists into array
# # documents as rows, unique words as columns (i.e., example as rows, features as columns)
# training_features = np.array([overlap_title, temp_diff, comm_auth, jaccard_sim]).T

# # scale
# training_features = preprocessing.scale(training_features)

In [23]:
print train_features.shape, new_features.shape, train_features[0]

(615513L, 10L) (615512L, 7L) [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]


In [24]:
training_features = np.concatenate((train_features[1:], new_features), axis=1)
print training_features.shape

(615512L, 17L)


In [1]:
print training_features[0]

NameError: name 'training_features' is not defined

In [30]:
# write features to .csv file
X_train = np.array(training_set[:]).T
X_features = training_features[:].T
features = zip(X_features[0], X_features[1], X_features[2], X_features[3], X_features[4], \
              X_features[5], X_features[6], X_features[7], X_features[8], X_features[9], X_features[10], X_features[11], \
              X_features[12], X_features[13], X_features[14], X_features[15], X_features[16])
with open("features_train_unscaled_authors.csv","wb") as feat:
    csv_out = csv.writer(feat)
    csv_out.writerow(('source', 'target', 'label', 'overlap_title', 'temp_diff', 'common_author', 'overlap_summary',\
                      'jaccard_sim', 'adamic', 'tfidf_cosine_sim', 'max_collab', 'mean_collab', 'max_cite', 'mean_cite', \
                      'avg_cite_A', 'avg_cite_B', 'indegree_target'))
    for row in features:
        csv_out.writerow(row)

In [84]:
training_features[]

array(['9510123.0', '9502114.0', '1.0', '2.0', '0.0', '0.0', '2.0',
       '0.0588235294118', '0.51389834237', '0.126084128029', '8'], 
      dtype='|S32')

### Computing features for test set:

In [28]:
#tfidf_dict_test = cPickle.load(open('tfidf_features_test.p', 'rb'))
#print type(tfidf_dict_test)

<type 'dict'>


In [21]:
### creating features and store in a csv file
max_collab_A_B = [] # max collaboration between authors of A and B
mean_collab_A_B = []

max_cite_A_B = [] # max citation by authors of A to B
mean_cite_A_B = []

#avg_cite_A_1 = [] # average citations authors of source have received
avg_cite_A_2 = [] # average citations authors of source have made

#avg_cite_B_1 = [] # average citations authors of target have received
avg_cite_B_2 = [] # average citations authors of target have made

indegree = []

counter = 0
for i in xrange(len(testing_set)):
    source = testing_set[i][0]
    target = testing_set[i][1]
    
    authors_A = source_info[3].split(',')
    authors_B = target_info[3].split(',')
    
    max_collab = []
    max_cite = []
    for auth_a in authors_A:
        a = auth_a.strip()
        for auth_b in authors_B:
            b = auth_b.strip()
            if a == '' or b == '':
                continue
            max_collab.append(collab[a][b])
            max_cite.append(cite[a][b])
            
    max_collab_A_B.append(0 if not len(max_collab) else max(max_collab))
    mean_collab_A_B.append(0 if not len(max_collab) else sum(max_collab)/len(max_collab))
    
    max_cite_A_B.append(0 if not len(max_cite) else max(max_cite) )
    mean_cite_A_B.append(0 if not len(max_cite) else sum(max_cite)/len(max_cite))
    
    avg_cite = []
    for auth_a in authors_A:
        a = auth_a.strip()
        if a == '' or not a in cite:
            continue
        avg_cite.append(sum(cite[a].values()))
    avg_cite_A_2.append(0 if not len(avg_cite) else sum(avg_cite)/len(avg_cite))
    
    avg_cite = []
    for auth_a in authors_B:
        a = auth_a.strip()
        if a == '' or not a in cite:
            continue
        avg_cite.append(sum(cite[a].values()))
    avg_cite_B_2.append(0 if not len(avg_cite) else sum(avg_cite)/len(avg_cite))
    
    indegree.append(G.in_degree(target)/len(authors_B))
    counter += 1
    if counter % 10000 == True:
        print counter, "testing examples processsed"

1 testing examples processsed
10001 testing examples processsed
20001 testing examples processsed
30001 testing examples processsed


In [22]:
new_features = np.array([max_collab_A_B, mean_collab_A_B, max_cite_A_B, mean_cite_A_B, avg_cite_A_2, avg_cite_B_2, indegree]).T
test_features = np.genfromtxt('features_test_unscaled.csv', delimiter=',')

In [23]:
print test_features.shape, new_features.shape

(32649L, 9L) (32648L, 7L)


In [24]:
testing_features = np.concatenate((test_features[1:], new_features), axis=1)
print testing_features.shape

(32648L, 16L)


In [27]:
print testing_features[0]

[  9.80707600e+06   9.80713900e+06   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   9.44587143e-02   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   2.51100000e+03   0.00000000e+00   3.00000000e+00]


In [28]:
# write features to .csv file
X_test = np.array(testing_set[:]).T
X_test_features = testing_features.T
features = zip(X_test_features[0], X_test_features[1], X_test_features[2], X_test_features[3], X_test_features[4], \
              X_test_features[5], X_test_features[6], X_test_features[7], X_test_features[8], X_test_features[9], \
              X_test_features[10], X_test_features[11], X_test_features[12], X_test_features[13], X_test_features[14],
              X_test_features[15])
with open("features_test_unscaled_authors.csv","wb") as feat:
    csv_out = csv.writer(feat)
    csv_out.writerow(('source', 'target', 'overlap_title', 'temp_diff', 'common_author', 'overlap_summary', 'jaccard_sim'\
                     ,'adamic', 'tfidf_cosine_sim', 'max_collab', 'mean_collab', 'max_cite', 'mean_cite', \
                      'avg_cite_A', 'avg_cite_B', 'indegree_target'))
    for row in features:
        csv_out.writerow(row)

In [42]:
# to delete
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import f1_score, accuracy_score

#print X.shape, y.shape, training_features.shape
X_train, X_val, y_train, y_val = train_test_split(
   training_features, y, test_size=0.01, random_state=42)

In [43]:
# to delete
clf = RandomForestClassifier(n_estimators=100, max_depth=15, n_jobs=4, max_features=8)
clf.fit(X_train, y_train)
y_val_pred = clf.predict(X_val)
f1score = f1_score(y_val, y_val_pred)
acc = accuracy_score(y_val, y_val_pred)
print 'F1-score: ', f1score
print 'Accuracy: ', acc

F1-score:  1.0
Accuracy:  1.0


In [ ]:
y_test_pred = clf.predict(X_test)

test_predictions_random = zip(range(len(y_test_pred)), y_test_pred)

with open("unscaled_randomforest_authors.csv","wb") as pred1:
    csv_out = csv.writer(pred1)
    csv_out.writerow(('ID','category'))
    for row in test_predictions_random:
        csv_out.writerow(row)